In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, List, Dict
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    temperature=0.3,
)

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)


C:\Users\emon1\AppData\Local\Temp\ipykernel_14864\2752009414.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\emon1\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLI

## Example 1: Detecting Factual Hallucinations

In [5]:
def test_hallucination_basic():
    """Test basic hallucination with factual questions"""
    
    questions = [
        "Who invented the telephone?",
        "What is the capital of Mars?",  # Nonsense question
        "When did World War II end?",
        "Who is the current king of United States?",  # US doesn't have a king
    ]
    
    print("="*60)
    print("TESTING FACTUAL HALLUCINATIONS")
    print("="*60)
    
    for i, question in enumerate(questions, 1):
        print(f"\n{i}. Question: {question}")
        response = llm.invoke([HumanMessage(content=question)])
        print(f"   Answer: {response.content}")
        print("-"*60)

# Run test
test_hallucination_basic()

TESTING FACTUAL HALLUCINATIONS

1. Question: Who invented the telephone?
   Answer: **Alexander Graham Bell** is widely credited with inventing the telephone and was granted the first U.S. patent for it on **March 7, 1876**.

His famous first words spoken over the device were to his assistant, Thomas A. Watson, on March 10, 1876: "Mr. Watson, come here, I want to see you."

However, the invention of the telephone is a complex story with several other individuals who were working on similar technologies around the same time and whose contributions are also recognized:

*   **Antonio Meucci:** An Italian immigrant who demonstrated a voice communication apparatus in New York in 1860 and filed a caveat (an intention to patent) in 1871, but couldn't afford to renew it. His work was recognized by the U.S. House of Representatives in 2002.
*   **Elisha Gray:** An American electrical engineer who filed a patent caveat for a telephone design on the very same day Bell filed his patent applicatio

## Example 2: Source-based Hallucination Detection

In [6]:
# Sample document
SAMPLE_DOCUMENT = """
Tesla Inc. is an American electric vehicle and clean energy company. 
The company was founded in 2003 by Martin Eberhard and Marc Tarpenning.
Elon Musk joined as chairman in 2004 and became CEO in 2008.
Tesla's main products include electric cars like Model S, Model 3, Model X, and Model Y.
The company's headquarters is located in Austin, Texas.
In 2023, Tesla delivered approximately 1.8 million vehicles worldwide.
"""

In [7]:
def check_source_hallucination(context: str, question: str, answer: str) -> dict:
    """Check if answer is grounded in the source"""
    
    # Method 1: Simple keyword matching
    answer_lower = answer.lower()
    context_lower = context.lower()
    
    # Extract key facts from answer
    # For demo, we'll use a simple approach
    
    # Method 2: Use LLM to verify
    verification_prompt = f"""
    Given the following context, verify if the answer is supported by the context.
    
    Context: {context}
    
    Question: {question}
    Answer: {answer}
    
    Respond with JSON:
    {{
        "is_supported": true/false,
        "confidence": 0-100,
        "explanation": "brief explanation",
        "unsupported_claims": ["list of claims not in context"]
    }}
    """
    
    verification = llm.invoke([HumanMessage(content=verification_prompt)])
    
    try:
        result = json.loads(verification.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"is_supported": False, "confidence": 0, "explanation": "Could not parse"}
    
    return result

In [8]:
print("="*60)
print("TESTING SOURCE-BASED HALLUCINATIONS")
print("="*60)

TESTING SOURCE-BASED HALLUCINATIONS


In [9]:
test_cases = [
    {
        "question": "Who founded Tesla?",
        "answer": "Tesla was founded by Martin Eberhard and Marc Tarpenning in 2003."
    },
    {
        "question": "When did Elon Musk join Tesla?",
        "answer": "Elon Musk joined Tesla in 2004 as chairman."
    },
    {
        "question": "How many cars did Tesla deliver in 2023?",
        "answer": "Tesla delivered approximately 3 million vehicles in 2023."  # WRONG!
    },
    {
        "question": "Where is Tesla's headquarters?",
        "answer": "Tesla's headquarters is in San Francisco, California."  # WRONG!
    }
]

In [10]:
for i, test_case in enumerate(test_cases, 1):
    print(f"\n{i}. Question: {test_case['question']}")
    print(f"   Answer: {test_case['answer']}")
    
    result = check_source_hallucination(
        SAMPLE_DOCUMENT,
        test_case['question'],
        test_case['answer']
    )
    
    print(f"   Supported: {result.get('is_supported', 'Unknown')}")
    print(f"   Confidence: {result.get('confidence', 0)}%")
    print(f"   Explanation: {result.get('explanation', 'N/A')}")
    
    if not result.get('is_supported', False):
        print(f"   Unsupported claims: {result.get('unsupported_claims', [])}")
    
    print("-"*60)


1. Question: Who founded Tesla?
   Answer: Tesla was founded by Martin Eberhard and Marc Tarpenning in 2003.
   Supported: True
   Confidence: 100%
   Explanation: The context explicitly states: 'The company was founded in 2003 by Martin Eberhard and Marc Tarpenning.' This directly matches the answer provided.
------------------------------------------------------------

2. Question: When did Elon Musk join Tesla?
   Answer: Elon Musk joined Tesla in 2004 as chairman.
   Supported: True
   Confidence: 100%
   Explanation: The context explicitly states: 'Elon Musk joined as chairman in 2004'. This directly supports both the year and the role mentioned in the answer.
------------------------------------------------------------

3. Question: How many cars did Tesla deliver in 2023?
   Answer: Tesla delivered approximately 3 million vehicles in 2023.
   Supported: False
   Confidence: 100%
   Explanation: The context states that Tesla delivered approximately 1.8 million vehicles in 2023, 

## Example 3: RAG with Hallucination Detection

In [11]:
# Create a sample knowledge base
knowledge_base = [
    "Python was created by Guido van Rossum and first released in 1991.",
    "JavaScript was created by Brendan Eich in 1995 in just 10 days.",
    "Java was developed by James Gosling at Sun Microsystems and released in 1995.",
    "C++ was developed by Bjarne Stroustrup starting in 1979.",
    "Go was designed by Robert Griesemer, Rob Pike, and Ken Thompson at Google.",
]

In [14]:
# Create vector store
docs = [Document(page_content=text) for text in knowledge_base]
vectorstore = Chroma.from_documents(docs, embeddings)

In [18]:
def rag_with_hallucination_check(query: str) -> dict:
    """RAG with built-in hallucination detection"""
    
    # Step 1: Retrieve relevant docs
    retrieved_docs = vectorstore.similarity_search(query, k=2)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    
    print(f"Retrieved Context:")
    print(context)
    print()
    
    # Step 2: Generate answer
    prompt = f"""Based on the following context, answer the question.
    If the answer is not in the context, say "I don't have enough information."
    
    Context: {context}
    
    Question: {query}
    
    Answer:"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    answer = response.content
    
    print(f"Generated Answer:")
    print(answer)
    print()
    
    # Step 3: Check for hallucination
    if "don't have enough information" in answer.lower() or "not in the context" in answer.lower():
        hallucination_risk = "LOW"
        grounded = True
    else:
        # Verify answer against context
        verification = check_source_hallucination(context, query, answer)
        grounded = verification.get('is_supported', False)
        hallucination_risk = "LOW" if grounded else "HIGH"
    
    return {
        "query": query,
        "context": context,
        "answer": answer,
        "grounded": grounded,
        "hallucination_risk": hallucination_risk
    }

In [19]:
# Test RAG
print("="*60)
print("RAG WITH HALLUCINATION DETECTION")
print("="*60)

RAG WITH HALLUCINATION DETECTION


In [15]:
test_queries = [
    "Who created Python?",  # In knowledge base
    "When was JavaScript created?",  # In knowledge base
    "What is the capital of France?",  # NOT in knowledge base
    "Who invented Rust?",  # NOT in knowledge base
]

In [20]:
for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    
    result = rag_with_hallucination_check(query)
    
    print(f"Grounded: {result['grounded']}")
    print(f"Hallucination Risk: {result['hallucination_risk']}")
    print()


Query: Who created Python?
Retrieved Context:
Python was created by Guido van Rossum and first released in 1991.
C++ was developed by Bjarne Stroustrup starting in 1979.

Generated Answer:
Guido van Rossum

Grounded: True
Hallucination Risk: LOW


Query: When was JavaScript created?
Retrieved Context:
JavaScript was created by Brendan Eich in 1995 in just 10 days.
Java was developed by James Gosling at Sun Microsystems and released in 1995.

Generated Answer:
JavaScript was created in 1995.

Grounded: True
Hallucination Risk: LOW


Query: What is the capital of France?
Retrieved Context:
C++ was developed by Bjarne Stroustrup starting in 1979.
Python was created by Guido van Rossum and first released in 1991.

Generated Answer:
I don't have enough information.

Grounded: True
Hallucination Risk: LOW


Query: Who invented Rust?
Retrieved Context:
C++ was developed by Bjarne Stroustrup starting in 1979.
Go was designed by Robert Griesemer, Rob Pike, and Ken Thompson at Google.

Generate

## Example 4: Self-Consistency Check

In [21]:
def self_consistency_check(question: str, num_samples: int = 3) -> dict:
    """Generate multiple answers and check consistency"""
    
    print(f"Question: {question}\n")
    
    answers = []
    
    # Generate multiple responses
    for i in range(num_samples):
        response = llm.invoke([HumanMessage(content=question)])
        answers.append(response.content)
        print(f"Answer {i+1}: {response.content}")
    
    # Check consistency using LLM
    consistency_prompt = f"""
    Compare these {num_samples} answers to the same question and determine:
    1. Are they consistent with each other?
    2. What are the differences?
    3. Which answer is most likely correct?
    
    Question: {question}
    
    Answers:
    {chr(10).join([f"{i+1}. {ans}" for i, ans in enumerate(answers)])}
    
    Respond with JSON:
    {{
        "consistent": true/false,
        "confidence": 0-100,
        "differences": ["list of differences"],
        "recommended_answer": "the most reliable answer",
        "hallucination_risk": "LOW/MEDIUM/HIGH"
    }}
    """
    
    consistency_result = llm.invoke([HumanMessage(content=consistency_prompt)])
    
    try:
        result = json.loads(consistency_result.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"consistent": False, "confidence": 0}
    
    print(f"\nConsistency Analysis:")
    print(f"   Consistent: {result.get('consistent', 'Unknown')}")
    print(f"   Confidence: {result.get('confidence', 0)}%")
    print(f"   Hallucination Risk: {result.get('hallucination_risk', 'Unknown')}")
    
    return result

In [22]:
# Test self-consistency
print("="*60)
print("SELF-CONSISTENCY CHECK")
print("="*60)

SELF-CONSISTENCY CHECK


In [23]:
test_questions = [
    "What is 25 * 47?",  # Factual, should be consistent
    "Who was the 100th president of the United States?",  # Nonsense, may be inconsistent
]

for question in test_questions:
    print(f"\n{'='*60}")
    result = self_consistency_check(question, num_samples=3)
    print()


Question: What is 25 * 47?

Answer 1: To calculate 25 * 47, you can use a couple of methods:

**Method 1: Standard Multiplication**
   47
 x 25
-----
  235  (5 * 47)
 940  (20 * 47)
-----
1175

**Method 2: Using the fact that 25 = 100/4**
25 * 47 = (100 / 4) * 47
= 100 * (47 / 4)
= 100 * 11.75
= 1175

The answer is **1175**.
Answer 2: To calculate 25 * 47, you can use a few methods:

**Method 1: Direct Multiplication**
```
   47
 x 25
 ----
  235  (5 * 47)
 940  (20 * 47)
 ----
 1175
```

**Method 2: Breaking down one number**
25 * 47 = 25 * (40 + 7)
= (25 * 40) + (25 * 7)
= 1000 + 175
= 1175

**Method 3: Using 25 = 100/4**
25 * 47 = (100 / 4) * 47
= (100 * 47) / 4
= 4700 / 4
= 1175

The answer is **1175**.
Answer 3: To calculate 25 * 47, you can do it in a few ways:

**Method 1: Standard Multiplication**
```
   47
 x 25
 ----
  235  (5 * 47)
 940  (20 * 47)
 ----
 1175
```

**Method 2: Breaking Down the Numbers**
You can think of 25 * 47 as 25 * (40 + 7):
*   25 * 40 = 1000 (since 25

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 53.829001504s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 53
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 51.478405881s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 51
}
]

## Example 5: Chain-of-Thought with Verification

In [24]:
def chain_of_thought_with_verification(question: str) -> dict:
    """Use CoT and then verify the reasoning"""
    
    # Step 1: Get answer with Chain of Thought
    cot_prompt = f"""
    Answer the following question step by step. Show your reasoning.
    
    Question: {question}
    
    Think through this carefully:
    1. What information do I need?
    2. What do I know?
    3. What is my reasoning?
    4. What is my final answer?
    """
    
    response = llm.invoke([HumanMessage(content=cot_prompt)])
    reasoning = response.content
    
    print(f"Chain of Thought Reasoning:")
    print(reasoning)
    print()
    
    # Step 2: Verify the reasoning
    verification_prompt = f"""
    Verify the following reasoning and answer. Point out any logical errors,
    unsupported claims, or potential hallucinations.
    
    Question: {question}
    
    Reasoning and Answer:
    {reasoning}
    
    Provide verification in JSON:
    {{
        "logical_errors": ["list of errors"],
        "unsupported_claims": ["claims without evidence"],
        "confidence_score": 0-100,
        "hallucination_detected": true/false,
        "corrected_answer": "if hallucination found, provide correct answer"
    }}
    """
    
    verification = llm.invoke([HumanMessage(content=verification_prompt)])
    
    try:
        result = json.loads(verification.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"hallucination_detected": False}
    
    print(f"Verification Results:")
    print(json.dumps(result, indent=2))
    
    return {
        "question": question,
        "reasoning": reasoning,
        "verification": result
    }

In [25]:
# Test CoT with verification
print("="*60)
print("CHAIN-OF-THOUGHT WITH VERIFICATION")
print("="*60)

CHAIN-OF-THOUGHT WITH VERIFICATION


In [26]:
test_questions = [
    "If a train travels 120 miles in 2 hours, what is its average speed?",
    "Who won the World Cup in 2050?",  # Future event
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print('='*60)
    result = chain_of_thought_with_verification(question)
    print()


Question: If a train travels 120 miles in 2 hours, what is its average speed?
Chain of Thought Reasoning:
Here's the step-by-step reasoning to find the average speed:

1.  **What information do I need?**
    To calculate average speed, I need two pieces of information:
    *   The total distance traveled.
    *   The total time taken to travel that distance.

2.  **What do I know?**
    From the question, I know:
    *   Total distance traveled = 120 miles.
    *   Total time taken = 2 hours.

3.  **What is my reasoning?**
    The formula for average speed is:
    Average Speed = Total Distance / Total Time
    I will substitute the known values into this formula and perform the division.

4.  **What is my final answer?**
    Average Speed = 120 miles / 2 hours
    Average Speed = 60 miles per hour

    The train's average speed is **60 mph**.



Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 15.392052721s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 15
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 13.039312549s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 13
}
]

## Example 6: Attribution-based Hallucination Detection

In [27]:
def attribution_check(context: str, claim: str) -> dict:
    """Check if a specific claim can be attributed to the context"""
    
    # Use embeddings to check similarity
    context_embedding = embeddings.embed_query(context)
    claim_embedding = embeddings.embed_query(claim)
    
    # Calculate cosine similarity
    similarity = cosine_similarity(
        [context_embedding],
        [claim_embedding]
    )[0][0]
    
    # Use LLM for detailed attribution
    attribution_prompt = f"""
    Context: {context}
    
    Claim: {claim}
    
    Can this claim be directly attributed to the context?
    If yes, quote the relevant part.
    If no, explain why.
    
    Respond with JSON:
    {{
        "attributed": true/false,
        "confidence": 0-100,
        "supporting_quote": "exact quote if found, else null",
        "explanation": "brief explanation"
    }}
    """
    
    response = llm.invoke([HumanMessage(content=attribution_prompt)])
    
    try:
        result = json.loads(response.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"attributed": False, "confidence": 0}
    
    result['embedding_similarity'] = float(similarity)
    
    return result

In [28]:
# Test attribution
print("="*60)
print("ATTRIBUTION-BASED HALLUCINATION DETECTION")
print("="*60)

ATTRIBUTION-BASED HALLUCINATION DETECTION


In [29]:
context = """
Apple Inc. announced its Q4 2023 earnings today. 
The company reported revenue of $89.5 billion, up 1% year over year.
iPhone sales contributed $43.8 billion to the revenue.
CEO Tim Cook stated that the company is excited about the Vision Pro launch.
"""

In [30]:
claims = [
    "Apple's Q4 2023 revenue was $89.5 billion",  # TRUE
    "iPhone sales were $43.8 billion",  # TRUE
    "Apple's revenue decreased by 5% year over year",  # FALSE
    "Apple announced a new MacBook Pro",  # NOT MENTIONED
]

In [31]:
for claim in claims:
    print(f"\nClaim: {claim}")
    result = attribution_check(context, claim)
    
    print(f"  Attributed: {result['attributed']}")
    print(f"  Confidence: {result['confidence']}%")
    print(f"  Embedding Similarity: {result['embedding_similarity']:.3f}")
    
    if result.get('supporting_quote'):
        print(f"  Quote: \"{result['supporting_quote']}\"")
    else:
        print(f"  Explanation: {result['explanation']}")
    
    print("-"*60)


Claim: Apple's Q4 2023 revenue was $89.5 billion


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 26.595633879s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 26
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 24.232310103s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 24
}
]

## Example 7: Confidence Calibration

In [32]:
def confidence_calibration(question: str) -> dict:
    """Ask model to provide confidence score with answer"""
    
    prompt = f"""
    Answer the following question and provide a confidence score (0-100).
    If you're not sure, be honest about it.
    
    Question: {question}
    
    Respond with JSON:
    {{
        "answer": "your answer",
        "confidence": 0-100,
        "reasoning": "why this confidence level",
        "sources_of_uncertainty": ["what makes you uncertain"]
    }}
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    try:
        result = json.loads(response.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"answer": response.content, "confidence": 50}
    
    return result

In [33]:
# Test confidence calibration
print("="*60)
print("CONFIDENCE CALIBRATION")
print("="*60)

CONFIDENCE CALIBRATION


In [34]:
test_questions = [
    "What is the capital of France?",  # Easy, high confidence
    "What is the population of Iceland?",  # Harder, medium confidence
    "What is the GDP of Bhutan?",  # Hard, low confidence
    "Who will win the next World Cup?",  # Impossible, should be very low
]

In [35]:
for question in test_questions:
    print(f"\nQuestion: {question}")
    result = confidence_calibration(question)
    
    print(f"  Answer: {result['answer']}")
    print(f"  Confidence: {result['confidence']}%")
    print(f"  Reasoning: {result.get('reasoning', 'N/A')}")
    
    if result['confidence'] < 50:
        print(f"  LOW CONFIDENCE - High hallucination risk!")
    
    print("-"*60)


Question: What is the capital of France?


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 37.801713196s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 37
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 35.429543893s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 35
}
]

## Best Practices to Reduce Hallucinations

In [ ]:
# ### 1 Prompt Engineering
# ```python
# Bad Prompt
# "Tell me about XYZ"
# 
# Good Prompt
# "Based on the provided context, tell me about XYZ. 
#  If the information is not in the context, say 'I don't know'."
# ```
# 
# ### 2 RAG (Retrieval-Augmented Generation)
# - Always provide context
# - Use vector databases
# - Verify answers against sources
# 
# ### 3 Self-Consistency
# - Generate multiple answers
# - Check for consistency
# - Flag inconsistencies
# 
# ### 4 Chain-of-Thought
# - Ask for step-by-step reasoning
# - Easier to spot logical errors
# - Verify each step
# 
# ### 5 Attribution
# - Require citations
# - Check if claims match sources
# - Use embedding similarity
# 
# ### 6 Confidence Scores
# - Ask model for confidence
# - Flag low-confidence answers
# - Human review for uncertain answers
# 
# ### 7 Human-in-the-Loop
# - Critical decisions need human verification
# - Use HITL for high-stakes applications
# 
# ### 8 Post-Processing
# - Verify facts against knowledge base
# - Use fact-checking APIs
# - Cross-reference multiple sources

## Complete Anti-Hallucination System

In [36]:
class AntiHallucinationSystem:
    """Complete system to detect and prevent hallucinations"""
    
    def __init__(self, llm, embeddings, knowledge_base=None):
        self.llm = llm
        self.embeddings = embeddings
        self.knowledge_base = knowledge_base
        
    def generate_with_checks(self, query: str, context: str = None) -> dict:
        """Generate answer with multiple hallucination checks"""
        
        results = {
            "query": query,
            "checks_performed": [],
            "warnings": [],
            "final_answer": None,
            "confidence": 0,
            "safe_to_use": False
        }
        
        # Check 1: Generate answer with context
        if context:
            prompt = f"""Based on this context, answer the question.
            If not in context, say "I don't have enough information."
            
            Context: {context}
            Question: {query}
            """
        else:
            prompt = query
        
        response = self.llm.invoke([HumanMessage(content=prompt)])
        answer = response.content
        
        results['initial_answer'] = answer
        results['checks_performed'].append("Initial generation")
        
        # Check 2: Self-consistency
        answers = [answer]
        for _ in range(2):
            resp = self.llm.invoke([HumanMessage(content=prompt)])
            answers.append(resp.content)
        
        # Simple consistency check
        all_similar = len(set(answers)) <= 2  # Allow minor variations
        results['checks_performed'].append(f" Self-consistency: {'PASS' if all_similar else 'FAIL'}")
        
        if not all_similar:
            results['warnings'].append(" Inconsistent answers detected")
        
        # Check 3: Source attribution (if context provided)
        if context:
            attribution = attribution_check(context, answer)
            results['checks_performed'].append(f"Attribution: {'PASS' if attribution['attributed'] else 'FAIL'}")
            
            if not attribution['attributed']:
                results['warnings'].append(" Answer not well-supported by context")
        
        # Check 4: Confidence calibration
        conf_result = confidence_calibration(query)
        confidence = conf_result.get('confidence', 50)
        results['confidence'] = confidence
        results['checks_performed'].append(f" Confidence: {confidence}%")
        
        if confidence < 60:
            results['warnings'].append(f"Low confidence ({confidence}%)")
        
        # Final decision
        if len(results['warnings']) == 0 and confidence >= 70:
            results['safe_to_use'] = True
            results['final_answer'] = answer
        elif len(results['warnings']) <= 1 and confidence >= 60:
            results['safe_to_use'] = True
            results['final_answer'] = f"{answer}\n(Note: Answer has medium confidence)"
        else:
            results['safe_to_use'] = False
            results['final_answer'] = "I cannot provide a reliable answer. Please verify with other sources."
        
        return results

In [37]:
# Test the complete system
print("="*60)
print("COMPLETE ANTI-HALLUCINATION SYSTEM")
print("="*60)

COMPLETE ANTI-HALLUCINATION SYSTEM


In [38]:
system = AntiHallucinationSystem(llm, embeddings)

In [39]:
context = """
The Moon is Earth's only natural satellite. It orbits Earth at an average
distance of 384,400 km. The Moon's diameter is 3,474 km, about one-quarter
of Earth's diameter. It takes 27.3 days to orbit Earth.
"""

In [40]:
test_cases = [
    ("What is the Moon's diameter?", context),
    ("How long does it take the Moon to orbit Earth?", context),
    ("What is the temperature on the Moon?", context),  # Not in context
]

In [41]:
for query, ctx in test_cases:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    
    result = system.generate_with_checks(query, ctx)
    
    print("\nChecks Performed:")
    for check in result['checks_performed']:
        print(f"  {check}")
    
    if result['warnings']:
        print("\nWarnings:")
        for warning in result['warnings']:
            print(f"  {warning}")
    
    print(f"\nConfidence: {result['confidence']}%")
    print(f"Safe to use: {result['safe_to_use']}")
    print(f"\nFinal Answer: {result['final_answer']}")


Query: What is the Moon's diameter?


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 23.190435606s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 23
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 20.813905344s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 20
}
]

In [42]:
# ## Summary & Takeaways
# 
# ## What We Learned:
# 
# 1. **Types of Hallucinations:**
#    - Factual hallucinations
#    - Source hallucinations
#    - Confidence hallucinations
# 
# 2. **Detection Methods:**
#    - Source verification
#    - Self-consistency checks
#    - Attribution checking
#    - Confidence calibration
#    - Chain-of-thought verification
# 
# 3. **Prevention Strategies:**
#    - Use RAG with proper context
#    - Implement multiple verification layers
#    - Request confidence scores
#    - Use self-consistency
#    - Employ human-in-the-loop
# 
### Key Principles:
# 
# **Always provide context** - Don't let the model guess
# **Verify against sources** - Check if answer is grounded
# **Use multiple checks** - Combine different methods
# **Request confidence** - Know when to trust the answer
# **Human review for critical** - Don't automate everything
# 
### Never Do:
# 
# Trust LLM output blindly for critical applications
# Use LLMs without context for factual questions
# Ignore low confidence scores
# Skip verification steps to save time
# 
### When to Use What:
# 
# | Use Case | Best Method |
# |----------|-------------|
# | Factual Q&A | RAG + Attribution |
# | Creative Writing | Self-Consistency |
# | Critical Decisions | Full System + HITL |
# | Simple Queries | Confidence Calibration |
# | Research | Multiple Sources + Verification |

In [43]:
print("Hallucination Tutorial Complete!")
print("\nYou now know:")
print(" - What hallucinations are and why they occur")
print(" - How to detect different types of hallucinations")
print(" - Multiple methods to reduce hallucinations")
print(" - How to build a complete anti-hallucination system")
print("\n You're ready to build reliable AI systems!")

Hallucination Tutorial Complete!

You now know:
 - What hallucinations are and why they occur
 - How to detect different types of hallucinations
 - Multiple methods to reduce hallucinations
 - How to build a complete anti-hallucination system

 You're ready to build reliable AI systems!
